In [15]:
import json
import dill as pkl
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

from arango import ArangoClient

In [2]:
# Establish Client Connection
client = ArangoClient(hosts='http://127.0.0.1:8530')

db = client.db('reddit_comp_db', username='root', password='compdb_kyle')

# Simple Queries

Lets start with some basic queries. Lets return:
    
    -The Number of Posts with >300 upvotes
    -The Number of Comments with >300 upvotes
    -The Number of Users with >5 comments
    -The top 5 posts (upvotes)
    -The top 5 users (post upvotes)


In [3]:
# Number of Posts with >300 Upvotes

query = """for post in posts
        filter post.num_upvotes > 300
        collect with count into upvotes
        return upvotes
        """

results = db.aql.execute(query)
print(list(results))

[170]


In [4]:
# Number of comments with >300 Upvotes

query = """for comment in comments
        filter comment.upvotes > 300
        collect with count into num_upvotes
        return num_upvotes
        """

results = db.aql.execute(query)
print(list(results))

[174]


In [5]:
# Number of users with >5 comments

query = """
        for doc in commented
        collect user = doc._from 
        aggregate num_comments = COUNT_DISTINCT(doc._to)
        filter num_comments > 5
        collect with count into num_users
        return num_users
        """

results = db.aql.execute(query)
print(list(results))

[339]


In [6]:
# top 5 posts 

query = """
        for post in posts
        sort post.num_upvotes desc
        limit 5
        return {[post._id]:post.num_upvotes}
        """
results = db.aql.execute(query)
print(list(results))

[{'posts/pst_176': 332}, {'posts/pst_444': 331}, {'posts/pst_445': 331}, {'posts/pst_446': 331}, {'posts/pst_447': 331}]


In [7]:
# top 5 users based on number of comments

query = """
        for doc in commented
        collect user = doc._from 
        aggregate num_comments = COUNT_DISTINCT(doc._to)
        sort num_comments desc
        limit 5
        return {[user]: num_comments}
        """
results = db.aql.execute(query)
print(list(results))

[{'users/VisualMod': 414}, {'users/AutoModerator': 191}, {'users/Dfree707': 64}, {'users/MoneyManToTheMoon': 44}, {'users/limethedragon': 40}]


# Graph Traversals 

Graph travelsals are where graph databases can provide the most value. For our database, we will investigate 2 questions that can be uniquely answered through graph traversals.

Questions:

    -How much of the information in the database can be traced back to the posts made by the AutoModerator
    -How many users are involved in a conversation about a given topic?

In [8]:
# First, lets establish how many posts are made by the AutoModerator

query = """
        for doc in posted
        collect user = doc._from 
        aggregate num_posts = COUNT_DISTINCT(doc._to)
        filter user == 'users/AutoModerator'
        return {[user]: num_posts}
        """
results = db.aql.execute(query)
print(list(results))

[{'users/AutoModerator': 22}]


In [9]:
# Call the reddit graph
reddit = db.graph('reddit')

In [10]:
query =  """for v,e,p in 3..6 any 'users/AutoModerator' graph reddit
            filter p.vertices[1].tag == 'Daily Discussion' && (LENGTH(p.vertices) <= 4 || p.vertices[4].title)
            filter v._id like 'users/%'
            return distinct(v)"""

results = db.aql.execute(query)
print(len(list(results)))


117


**There are 117 users who are involved around ideas from 'Daily Discussion'**

The analysis can be explained as followed:

    -All Users who comment on the daily discussion posts are included in the Daily Discussion community
    -If one of the users who comments also makes a post, anyone who comments on their post is included in the community
    -The assumption here is that commenters are taking sharing their same opinions from the Daily Discussion and creating      posts.
    
## Traversal 2

**Find how many users are in conversations involving gamestop**

The criteria for the above question is as follows:

    -Any user who posts or comments about gamestop
    -Any user who comments on a post about gamestop
    -Any user who comments in a comment thread about gamestop

In [11]:
# Call the reddit with threads graph
reddit_w_threads = db.graph('reddit_w_threads')

In [12]:
#Lets start by compiling a list of all the comments and posts that 

query = """
        for comment in comments
        filter comment.body =~ "(?i)gme|gamestop"
        return comment._id
        """
results = db.aql.execute(query)
gme_comments = list(results)

query = """
        for post in posts
        filter post.body =~ "(?i)gme|gamestop"
        return post._id"""

results = db.aql.execute(query)
gme_posts = list(results)

In [13]:
# First we can loop through the posts and return the post author and comment authors
users_from_post_comments = []
for post in gme_posts:
    query = f"""
            for v,e,p in 1..2 any '{post}' graph reddit_w_threads
            filter p.edges[1]._id like 'commented%' || length(p.verticies) < 3
            filter v._id like 'users/%'
            return distinct(v._key)
            """
    results = db.aql.execute(query)
    users_from_post_comments = users_from_post_comments + list(results)

In [14]:
# Next we wil loop through all the comments and return the comment author and authors involved in the thread
users_from_comment_threads = []
for comment in gme_comments:
    query = f"""
            for v,e,p in 1..3 any '{comment}' graph reddit_w_threads
            filter (p.edges[0]._id like "commented/%" && length(p.edges) == 1) || 
            (p.edges[0]._id like "comment_thread/%" && p.edges[1]._id like "commented/%" && not p.edges[2]._id) ||
            (p.edges[0]._id like "comment_thread/%" && p.edges[1]._id like "comment_thread/%"  && p.edges[2]._id like "commented/%")
            filter v._id like 'users/%'
            return distinct(v._key)
            """
    results = db.aql.execute(query)
    users_from_comment_threads = users_from_comment_threads + list(results)

In [16]:
# Finally we will combine these lists and keep only unique values. This will give us all the unique users
# involved in gamestop conversations

all_involved_users = users_from_post_comments + users_from_comment_threads
all_involved_users_unique = np.unique(all_involved_users)

print(f'There are {len(all_involved_users_unique)} unique users involved in gamestop conversations')

There are 1537 unique users involved in gamestop conversations


In [20]:
query = """for user in users
        collect with count into num_users
        return num_users
        """

results = db.aql.execute(query)
num_users = list(results)

print(f"{(len(all_involved_users_unique)/num_users[0])*100}% of all users are involved in GME conversations")

22.379149679673848% of all users are involved in GME conversations


In [21]:
# Had we not used traversals to find users involved in coversations that had not mentioned gamestop, what would our 
# results have been?

only_direct_users = gme_posts + gme_comments
only_direct_users_unique = np.unique(only_direct_users)

print(f'There are {len(only_direct_users_unique)} unique users who mentioned gamestop')
print(f"{(len(only_direct_users_unique)/num_users[0])*100}% of all users mentioned in GME their post/comment")

There are 563 unique users who mentioned gamestop
8.197437390797903% of all users mentioned in GME their post/comment
